In [2]:
import pandas as pd 

In [3]:
corpus_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTest.csv")
#corpus_test

In [4]:
corpus_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corpusTrain.csv")
#corpus_train

In [5]:
corpus_test['Content'] = corpus_test['Content'].str.strip()
corpus_test['Content'] = corpus_test['Content'].str.lower()

corpus_train['Content'] = corpus_train['Content'].str.strip()
corpus_train['Content'] = corpus_train['Content'].str.lower()

In [6]:
spec_chars = ["!", '"', "#", "%", "&", "'", "(", ")", "*", "+", ",", "-", ".", "/", ":", ";", "<",
              "=", ">", "?", "@", "[", "\\", "]", "^", "_", "`", "{", "|", "}", "~", "–"]

for char in spec_chars:
    corpus_test['Content'] = corpus_test['Content'].str.replace(char, '')

for char in spec_chars:
    corpus_train['Content'] = corpus_train['Content'].str.replace(char, '')

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_test['Content'])
vectorized_test_content = vectorizer.transform(corpus_test['Content'])
print(vectorized_test_content.shape)
print(vectorized_test_content[0])

In [ ]:
print(vectorized_test_content[1])

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 1000)
vectorizer.fit(corpus_train['Content'])
vectorized_train_content = vectorizer.transform(corpus_train['Content'])
print(vectorized_train_content.shape)
print(vectorized_train_content[0])

In [ ]:
corpus_compined = pd.DataFrame(corpus_test['Content'].append(corpus_train['Content'], ignore_index = True)) 
corpus_compined['Content']

In [ ]:
vectorizer = CountVectorizer(lowercase = True, stop_words = 'english', max_features = 10000)
vectorizer.fit(corpus_compined['Content'])
vectorized_combined_content = vectorizer.transform(corpus_compined['Content'])
print(vectorized_combined_content.shape)

Exact Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
count = 0
for i in range(5374, 537364):
  for j in range(5374):
    c = cosine_similarity(vectorized_combined_content[j], vectorized_combined_content[i])
    if c > 0.8: 
      count = count + 1
print(count)

Exact Jaccard Similarity

In [ ]:
import numpy as np
from sklearn.metrics import jaccard_score

In [31]:
vectorized_test_arr = vectorized_combined_content[0:5374].toarray()

In [ ]:
vectorized_train_arr = vectorized_combined_content[5374:].toarray()

In [ ]:
count = 0
for i in range(5374, 537364):
  for j in range(5374):
    c = jaccard_score(vectorized_train_arr[i], vectorized_test_arr[j])
    if c > 0.8: 
      count = count + 1
print(count)

MinHash LSH

In [ ]:
!pip install --upgrade datasketch

In [50]:
from datasketch import MinHash, MinHashLSH

In [ ]:
num_permutations = 32
count = 0
for i in range(5374, 537364):
  set1 = set(corpus_train['Content'][i].split(" "))
  m1 = MinHash(num_perm=num_permutations)

  for d in set1:
      m1.update(d.encode('utf8'))

  for j in range(5374):
    set2 = set(corpus_test['Content'][j].split(" "))
    m2 = MinHash(num_perm=num_permutations)

    for d in set2:
        m2.update(d.encode('utf8'))

    # Create LSH index
    lsh = MinHashLSH(threshold=0.5, num_perm=num_permutations)
    lsh.insert("m2", m2)
    result = lsh.query(m1)
    if len(result) > 0: 
      count = count + 1
    else: 
      #remove the current row from the test corpus 
print(count)

LSH-Cosine

In [ ]:
from sklearn.neighbors import LSHForest

In [36]:
train_indexes_arr = []
for j in range(5374, 537364):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  train_indexes_arr.append(temp)

In [ ]:
test_indexes_arr = []
for j in range(5374):
  temp = []
  for i in range(len(vectorized_combined_content[j].nonzero()[0])):
    temp.append(vectorized_combined_content[j].nonzero()[1][i])
  test_indexes_arr.append(temp)

In [ ]:
lshf = LSHForest()
lshf.fit(train_indexes_arr) 
distances, indices = lshf.kneighbors(test_indexes_arr, n_neighbors=2)

LSH-Jaccard

In [ ]:
#updated arrays, contain only the test rows with lsh similarity > 0.5
count = 0
for i in range(5374, 537364):
  for j in range(5374):
    c = jaccard_score(vectorized_train_arr[i], vectorized_test_arr[j])
    if c > 0.8: 
      count = count + 1
print(count)